# ICESat2 IT Model Validation for 3 months of PCE data

## Basic GEODYN Run

In [1]:
run_list = ['msis2', 'tiegcm_oc', 'dtm2020','jb2008', ] 

In [ ]:
%load_ext autoreload
%autoreload 2

import sys  
import pickle 
import gc

sys.path.insert(0, '/data/geodyn_proj/pygeodyn/pygeodyn_develop/')
from PYGEODYN import Pygeodyn

for i,val in enumerate(run_list):   
    for ii in ['_p1', '_p2']:


        run_settings = '/data/zach_work/validation/ICESat2_3month_Validation/runsettings_3month_bwdrag_'+ val +ii+'.yaml'
        ### Load the data into an object
        RunObject = Pygeodyn(run_settings)
        RunObject.RUN_GEODYN()
        RunObject = 0
        gc.collect()



Run # 1     Current Time =      16:14:27  GMT-7
Run # 1
+=======================================================================
|     Running GEODYN with TIEGCM Orbit Cloud Method     
|                                                       
|     Running MSIS2 through GEODYN to construct a initialized orbit
Run # 1     Current Time =      16:14:30  GMT-7
Run # 1
+ ————————————————————————————————————————————————————————————————————————————————————————
|
| ---------------------- RUN PARAMETERS  ----------------------
|
|  Run # 1     IISSET Cleaned      tmp/.../cleaned_setup_2018292.1
|  Run # 1     Density Model:      msis2
|  Run # 1     GEODYN Version:     CD_model_proj
|  Run # 1     ARC run:            icesat2_2018292.1_24hr.msis2.bwdrag
|  Run # 1     Output directory:   /data/zach_work/output_from_runs/msis2/msis2_BWDRAG
|  Run # 1     EXAT File:          /data/data_geodyn/inputs/icesat2/external_attitude/EXAT01.2018.292.gz
|
|  Run # 1     Epoch Start:  2018-10-19 00:00:00
|  

In [ ]:
# sys.exit(0)

##  GEODYN read and dump to pickle, OR load pickle

In [ ]:
%load_ext autoreload
%autoreload 2

import sys  
import os
import pickle 
import gc

sys.path.insert(0, '/data/geodyn_proj/pygeodyn/pygeodyn_develop/')
from PYGEODYN import Pygeodyn

Obj_Geodyn = {}

for i,val in enumerate(run_list): #,'jb2008','msis2'
    for ii in ['_p1', '_p2']:
        
        dir_save = '/data/zach_work/output_from_runs/icesat2_3monthrun_pickles/'
        pickle_file = dir_save+'Fulltime_valid_'+val+ii+'.pkl'

        if not os.path.exists(pickle_file):

            print('Must create pickle file...')
            print('   ',  pickle_file)
            print('   ', 'Reading Geodyn Data')

            run_settings = '/data/zach_work/validation/ICESat2_3month_Validation/runsettings_3month_bwdrag_'+ val +ii+'.yaml'\
            ### Load the data into an object
            Obj_Geodyn = Pygeodyn(run_settings)
            Obj_Geodyn.getData_BigData_lowmemory()

            gc.collect()

            #### Pickle the object to save it
            print('   ', 'Saving pickle')
            filehandler = open(pickle_file, 'wb') 
            pickle.dump(Obj_Geodyn, filehandler)
            filehandler.close()
            Obj_Geodyn = 0
            print('   ', 'Saved pickle')


        else:
            filehandler = open(pickle_file, 'rb') 
            Obj_Geodyn[val] = pickle.load(filehandler)
            filehandler.close()

In [ ]:
# gc.collect()


In [ ]:
Obj_Geodyn = {}

for i,val in enumerate(run_list): #,'jb2008','msis2'
    for ii in ['_p1', '_p2']:
        
        dir_save = '/data/zach_work/output_from_runs/icesat2_3monthrun_pickles/'
        pickle_file = dir_save+'Fulltime_valid_'+val+ii+'.pkl'
        filehandler = open(pickle_file, 'rb') 
        Obj_Geodyn[val+ii] = pickle.load(filehandler)
        filehandler.close()

## Plots

In [ ]:
import plotly.graph_objects as go
from plotly.offline import plot, iplot
from plotly.subplots import make_subplots
import plotly.express as px
import pandas as pd
import datetime
import numpy as np

config = dict({
                'displayModeBar': True,
                'responsive': False,
                'staticPlot': False,
                'displaylogo': False,
                'showTips': False,
                })



import numpy as np
import pandas as pd
import os

# px.colors.colorscale_to_colors()
# plotly.colors.PLOTLY_SCALES["Viridis"]

def get_color(colorscale_name, loc):
    from _plotly_utils.basevalidators import ColorscaleValidator
    # first parameter: Name of the property being validated
    # second parameter: a string, doesn't really matter in our use case
    cv = ColorscaleValidator("colorscale", "")
    # colorscale will be a list of lists: [[loc1, "rgb1"], [loc2, "rgb2"], ...] 
    colorscale = cv.validate_coerce(colorscale_name)
    
    if hasattr(loc, "__iter__"):
        return [get_continuous_color(colorscale, x) for x in loc]
    return get_continuous_color(colorscale, loc)
        

# Identical to Adam's answer
import plotly.colors
from PIL import ImageColor

def get_continuous_color(colorscale, intermed):
    """
    Plotly continuous colorscales assign colors to the range [0, 1]. This function computes the intermediate
    color for any value in that range.

    Plotly doesn't make the colorscales directly accessible in a common format.
    Some are ready to use:
    
        colorscale = plotly.colors.PLOTLY_SCALES["Greens"]

    Others are just swatches that need to be constructed into a colorscale:

        viridis_colors, scale = plotly.colors.convert_colors_to_same_type(plotly.colors.sequential.Viridis)
        colorscale = plotly.colors.make_colorscale(viridis_colors, scale=scale)

    :param colorscale: A plotly continuous colorscale defined with RGB string colors.
    :param intermed: value in the range [0, 1]
    :return: color in rgb string format
    :rtype: str
    """
    if len(colorscale) < 1:
        raise ValueError("colorscale must have at least one color")

    hex_to_rgb = lambda c: "rgb" + str(ImageColor.getcolor(c, "RGB"))

    if intermed <= 0 or len(colorscale) == 1:
        c = colorscale[0][1]
        return c if c[0] != "#" else hex_to_rgb(c)
    if intermed >= 1:
        c = colorscale[-1][1]
        return c if c[0] != "#" else hex_to_rgb(c)

    for cutoff, color in colorscale:
        if intermed > cutoff:
            low_cutoff, low_color = cutoff, color
        else:
            high_cutoff, high_color = cutoff, color
            break

    if (low_color[0] == "#") or (high_color[0] == "#"):
        # some color scale names (such as cividis) returns:
        # [[loc1, "hex1"], [loc2, "hex2"], ...]
        low_color = hex_to_rgb(low_color)
        high_color = hex_to_rgb(high_color)

    return plotly.colors.find_intermediate_color(
        lowcolor=low_color,
        highcolor=high_color,
        intermed=((intermed - low_cutoff) / (high_cutoff - low_cutoff)),
        colortype="rgb",
    )




cols = get_color("Viridis", np.linspace(0, 1, 5))
map_cols = np.linspace(0, 1, 5)
colorscale=[]
for i,val in enumerate(map_cols):
    colorscale.append([val, cols[i]])
    



# Simplify Plotting Schemes:
col1 =  px.colors.qualitative.Plotly[2]
col2 =  px.colors.qualitative.Plotly[4]
col3 =  px.colors.qualitative.Plotly[1]
col4 =  px.colors.qualitative.Plotly[3]
col5 =  px.colors.qualitative.Plotly[4]
col6 =  px.colors.qualitative.Plotly[5]



# Simplify Plotting Schemes:
col_msis2 =  px.colors.qualitative.Plotly[2]
col_jb2008 =  px.colors.qualitative.Plotly[4]
col_dtm2020 =  px.colors.qualitative.Plotly[1]
col_tiegcm_oc =  px.colors.qualitative.Plotly[5]

x_annot_val = 1.1



def get_plot_params(plot_num, model_name_string):
    '''
    INPUT:   
        Plot number, model_name string, x_annot_val
    
    RETURN:
        col
        x_annot
        y_annot1
        y_annot2
        m_size
        
    '''
    
    
    if plot_num == 0:
        col = col1
        x_annot = x_annot_val
        y_annot1 = 1
        y_annot2 = .9
        m_size = 3.5
    elif plot_num == 1:
        x_annot = x_annot_val
        y_annot1 = .8
        y_annot2 = .8
        col = col2
        m_size = 3.5
    elif plot_num == 2:
        x_annot = x_annot_val
        y_annot1 = .5
        y_annot2 = .7
        col = col3
        m_size = 3.5
    elif plot_num == 3:
        x_annot = x_annot_val
        y_annot1 = .2
        y_annot2 = .55
        col = col4
        m_size = 3.5
    elif plot_num == 4:
        x_annot = x_annot_val
        y_annot1 =  0 
        y_annot2 = .45
        col = col5
        m_size = 3.5
        
    elif plot_num == 5:
        x_annot = x_annot_val
        y_annot1 =  0 
        y_annot2 = .35
        col = col6
        m_size = 3.5
        
    if model_name_string == 'msis2':
        col=col_msis2
    elif model_name_string == 'dtm2020':
        col=col_dtm2020
    elif model_name_string == 'jb2008':
        col=col_jb2008
    elif model_name_string == 'tiegcm_oc':
        col=col_tiegcm_oc
        
    ### Old Models
    elif model_name_string == 'dtm87':
        col='grey'
    elif model_name_string == 'jaachia71':
        col='grey'
    elif model_name_string == 'msis86':
        col='grey'
    elif model_name_string == 'msis00':
        col='tan'

        
        
    return(col,x_annot,y_annot1,y_annot2,m_size)
        

### F10.7 and Kp

In [ ]:
from netCDF4 import Dataset


def read_nc_file( filename, variables):
    ''' This function reads the TIEGCM .nc files and saves the given input variables to a dictionary.
        The breakloop feature is here so that if the file doesn't exist the code can still continue.  '''
    status = os.path.exists(filename)
    
    if status == True:
        data = {}
        for i, var_names in enumerate(variables):
            ncid =  Dataset(filename,"r+", format="NETCDF4")# filename must be a string
            varData = ncid.variables
            data[var_names] = np.array(varData[var_names])  
    elif status == False:
        print('No File Found', filename )
        breakloop = True
        data = 0
        return( data , breakloop)
    breakloop = False
    return(data,breakloop )



path_to_f107 = '/data/data_geodyn/gpi_1960001-2021243_f107aDaily.nc'
variables = ['year_day', 'f107d', 'f107a', 'kp']
f107_data = read_nc_file(path_to_f107, variables)

date = []
kp_list = []
f107d_list = []
f107a_list  = []


arc_list = [        2018287,
                    #
                    2018292,
                    2018293,
                    2018294,
                    2018295,
                    2018296,
                    2018297,
                    2018298,
                    2018299,                   
                    #
                    2018304,
                    2018305,
                    2018306,
                    2018307,
                    2018308,
                    #
                    2018313,
                    2018314,
                    2018315,
                    2018316,
                    2018317,
                    2018318,
                    2018319,
                    2018320,
                    2018321,
                    2018322,
                    2018323,
                    2018324,
                    2018325,
                    2018326,
                    2018327,
                    #
                    2018335,
                    2018336,
                    2018337,
                    #
                    2018349,
                    2018350,
                    2018351,
                    2018352,
                    #
                    2018356,
                    2018357,
                    2018358,
                    #
                    2018365,
                    2019001,
                    2019002,
                    2019003,
                    2019004,
                    2019005,
                    2019006,
                    2019007,
                    2019008,
                    ]


date = []
kp_list = []
f107d_list = []
f107a_list  = []
date_3hr = []


for i,val in enumerate(arc_list):
    
    index = f107_data[0]['year_day']==val
    kp_list.append(f107_data[0]['kp'][index][0])
    f107d_list.append(f107_data[0]['f107d'][index][0])
    f107a_list.append(f107_data[0]['f107a'][index][0])
    
    date.append(pd.to_datetime( str(val), format='%Y%j'))

    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=0))
    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=3))
    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=6))
    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=9))
    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=12))
    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=15))
    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=18))
    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=21))
#     date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=24))
    
kp_expand = []
for i in kp_list:
    for ii in i:
        kp_expand.append(ii)
        
        
        
solar_fluxes = {}

solar_fluxes['f107d_list'] = f107d_list
solar_fluxes['f107a_list'] = f107a_list
solar_fluxes['date']       = date
solar_fluxes['date_3hr']   = date_3hr
solar_fluxes['kp_expand']  = kp_expand
        
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=([ 'Kp','F10.7']),
    vertical_spacing = 0.2)





fig.add_trace(go.Scattergl(  x=date,
                             y=f107d_list,
                             name= 'F107d',
                             mode='markers',
                             opacity=1,
                             marker=dict(
                                color='grey', 
                                size=8,
                                        ),
                             showlegend=True,
                          ),
                              secondary_y=False,
                               row=2, col=1,
                          )

fig.add_trace(go.Scattergl(  x=date,
                             y=f107a_list,
                             name= 'F107a',
                             mode='markers',
                             opacity=1,
                             marker=dict(
                                color='black', 
                                size=8,
                                        ),
                             showlegend=True,
                          ),
                              secondary_y=False,
                               row=2, col=1,
                          )


fig.add_trace(go.Scattergl(  x=date_3hr,
                             y=kp_expand,
                             name= 'Kp',
                             mode='markers',
                             opacity=1,
                             marker=dict(
                                color='blue', 
                                size=8,
                                        ),
                             showlegend=True,
                          ),
                              secondary_y=False,
                               row=1, col=1,
                          )

fig.update_layout(title= 'NGDC Geophysical Indices -- F10.7 and Kp',
                autosize=True,
                font=dict(size=14),
                legend= {'itemsizing': 'constant'})

fig.update_yaxes( title="sfu", row=2, col=1)
fig.update_yaxes( title="Kp", row=1, col=1)
font_dict=dict(family='sans-serif',size=16,color='black')
fig.update_xaxes(showline=True,
                 showticklabels=True,
                 linecolor='black',
                 linewidth=1,
                 ticks='inside',
                 tickfont=font_dict,
                 mirror='allticks',
                 tickwidth=1.4,
                 tickcolor='black',
                 gridcolor='lightgray')


# fig.update_yaxes(range=[1.15, 1.35], row=1,col=1)
# fig.update_yaxes(range=[1.15, 1.35], row=1,col=2)
fig.update_yaxes(showline=True,      # add line at x=0
                 linecolor='black',  # line color
                 linewidth=1,      # line size
                 ticks='inside',    # ticks outside axis
                 tickfont=font_dict, # tick label font
                 mirror='allticks',  # add ticks to top/right axes
                 tickwidth=1.4,      # tick width
                 tickcolor='black',  # tick color
                  gridcolor='lightgray')

fig.update_annotations(font_size=18)  # Increase size of subplot title
fig.update_layout(
#                   title = '',
                  autosize=True,#    width=1000,    height=450,
                  legend= {'itemsizing': 'constant'},
                  font=font_dict,
                  plot_bgcolor='white', 
                 )
fig.show(config=config)



### Density

In [ ]:



def orb_avg(den_df, arc):
    
    
    #### Find the index for the correct date
    vals  = np.arange(den_df[arc].index[0],den_df[arc].index[-1]+1)
    df = den_df[arc].set_index('Date',drop=False ) 
    df['i_vals'] = vals
    index_date = df.loc[df.index.max()]['i_vals'].min()
    
#     print('index_date', index_date)
    lat = np.asarray(den_df[arc]['Lat'][:index_date])
    time_pd = pd.to_datetime(den_df[arc]['Date'][:index_date])
    i = np.nonzero( lat[1:]*lat[0:-1]  <  np.logical_and(0 , lat[1:] > lat[0:-1] )  )
    i = i[0]

    d_avg = np.zeros(np.size(i))
    height_avg = np.zeros(np.size(i))
    
#     print('time_pd',time_pd)

    time_avg = []
    d_avg_rolling = []
    
    roll_avg_count = 0
    for j in range(np.size(i)-1):
        d_avg[j]      = np.mean(den_df[arc]['rho (kg/m**3)'  ][i[j] : i[j+1]-1  ]  )
        height_avg[j] = np.mean(den_df[arc]['Height (meters)'][i[j] : i[j+1]-1  ]  )
#         mean_time      = np.mean(time_pd[   i[j] : i[j+1]-1  ])
        t1 = pd.to_datetime(time_pd[ i[j]    ])
        t2 = pd.to_datetime(time_pd[ i[j+1]-1])
        datemiddle = pd.Timestamp(t1) + (pd.Timestamp(t2) - pd.Timestamp(t1)) / 2

        time_avg.append(datemiddle)

        if roll_avg_count ==1:
            d_avg_rolling.append(np.mean([ d_avg[j],  d_avg[j-1]]))
            roll_avg_count =0
            
        roll_avg_count+=1 
    d_avg_rolling.append(np.mean([ d_avg[j],  d_avg[j-1]]))
        
    return(time_avg, d_avg, d_avg_rolling )
    

def plot_density_orbit_avg(fig, obj_m1, plot_num ):
    

    ####  Get plot Parameters for this model
    model_m1 = obj_m1.__dict__['global_params']['den_model']
    col,x_annot,y_annot1,y_annot2,m_size = get_plot_params(plot_num, model_m1)
    
    for ii,arc in enumerate(obj_m1.__dict__['global_params']['arc_input']):
        
        vals  = np.arange(obj_m1.__dict__['Density'][arc].index[0],obj_m1.__dict__['Density'][arc].index[-1]+1)
        df = obj_m1.__dict__['Density'][arc].set_index('Date',drop=False ) 
        df['i_vals'] = vals
        index_date = df.loc[df.index.max()]['i_vals'].min()

        
        time_avg,d_avg, d_avg_rolling = orb_avg(obj_m1.Density, arc)
        
        
        fig.add_trace(go.Scattergl(x=time_avg,
                                 y=d_avg_rolling,
#                                  y=d_avg,
                                name= model_m1 ,
                                mode='markers',
                                marker=dict(
                                color=col,
                                size=7,),
                                showlegend=False,
                                   ),
                                   row=1, col=1,
                                   )
        

        fig.update_yaxes(type="log", exponentformat= 'power',row=1, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_yaxes(title_text="kg/m^3", row=1, col=1)
#     fig.update_yaxes(title_text="nT", row=2, col=1)
#     fig.update_yaxes(title_text="sfu", row=3, col=1)
    fig.update_layout(legend= {'itemsizing': 'constant'})
#     fig.update_layout(
#         font=dict(          size=18, ),
#         autosize=False,
#         width=900,
#         height=1000,)
    
    return(fig)



def Plot_Densitycomparison(fig, obj_m1, plot_num):


    ####  Get plot Parameters for this model
    model_m1 = obj_m1.__dict__['global_params']['den_model']
    col,x_annot,y_annot1,y_annot2,m_size = get_plot_params(plot_num, model_m1)
    
    
    for ii,arc in enumerate(obj_m1.__dict__['global_params']['arc_input'][:]):
        
        
        #### INDEX THE DENSITY DF correctly
        vals  = np.arange(obj_m1.__dict__['Density'][arc].index[0],obj_m1.__dict__['Density'][arc].index[-1]+1)
        df = obj_m1.__dict__['Density'][arc].set_index('Date',drop=False ) 
        df['i_vals'] = vals
        index_date = df.loc[df.index.max()]['i_vals'].min()

        
        str_run_param = 'run_parameters'+ arc
        final_iter = obj_m1.__dict__[str_run_param]['str_iteration']
        i_arc = ii+1
        
        
        
        vals  = np.arange(obj_m1.__dict__['Density'][arc].index[0],obj_m1.__dict__['Density'][arc].index[-1]+1)
        df = obj_m1.__dict__['Density'][arc].set_index('Date',drop=False ) 
        df['i_vals'] = vals
        index_date = df.loc[df.index.max()]['i_vals'].min()       
        time_avg,d_avg, d_avg_rolling = orb_avg(obj_m1.Density, arc)
        
        
        
        print('----',model_m1,'----')
        print('     mean:    ',np.mean(obj_m1.Density[arc]['rho (kg/m**3)']),'----')
        print('     variance:',np.std(obj_m1.Density[arc]['rho (kg/m**3)']),'----')
        print()
#         if ii ==0:
#             fig.add_trace(go.Scattergl(  x=obj_m1.Density[arc]['Date'][:index_date][:],
#                                      y=obj_m1.Density[arc]['rho (kg/m**3)'][:index_date][:],
#                                      name= model_m1,
#                                      mode='markers',
#                                      opacity=1,
#                                      marker=dict(
#                                         color=col, 
#                                         size=m_size,),
#                                      showlegend=True,),
#                                       secondary_y=False,
#                                        row=1, col=1,)

#         else:
#             fig.add_trace(go.Scattergl(  x=obj_m1.Density[arc]['Date'][:index_date][:],
#                          y=obj_m1.Density[arc]['rho (kg/m**3)'][:index_date][:],
#                          name= model_m1,
#                          mode='markers',
#                          opacity=1,
#                          marker=dict(
#                             color=col, 
#                             size=m_size,),
#                          showlegend=False,),
#                           secondary_y=False,
#                            row=1, col=1,)
        fig.add_trace(go.Scattergl(x=time_avg,
                     y=d_avg_rolling,
                    name= model_m1 ,
                    mode='markers',
                    marker=dict(
                    color='black',
                    size=5,),
                    showlegend=False,),
                       row=1, col=1,)


#         (mean,rms,rms_about_zero) = STATS_residuals(data_resids['T'], 'in-track')

#         fig = add_stats_annotation(fig, model_m1+'<br>Mean='+ str(np.round(mean,4))+'<br>RMS='+ str(np.round(rms_about_zero,4)), col , x_annot, y_annot1)


    fig.update_yaxes( title=r"$\frac{kg}{m^3}$", type='log', exponentformat= 'power',row=1, col=1)
    fig.update_xaxes( title="Date", row=1, col=1)

    return(fig)


def legend_as_annotation(fig, den_model_string, color_it, x_annot, y_annot):
    fig.add_annotation(
            x=x_annot,
            y=y_annot,
            xref="paper",
            yref="paper",
            showarrow=False,
            text=den_model_string,
            font=dict(
                size=16,
                color="#ffffff"
                ),
            align="center",
            bordercolor="#c7c7c7",
            borderwidth=2,
            borderpad=4,
            bgcolor=color_it,
            opacity=0.9
            )

    return(fig)

def add_stats_annotation(fig, text_in, col, x_annot, y_annot):
    fig.add_annotation(
            x=x_annot,
            y=y_annot,
            xref="x domain",
            yref="y domain",
            showarrow=False,
            text=text_in,
            font=dict(
                size=13,
                color="#ffffff"
                ),
            align="center",
            bordercolor="#c7c7c7",
            borderwidth=2,
            borderpad=4,
            bgcolor=col,
            opacity=0.9
            )
    return(fig)

import pandas as pd

def STATS_residuals(residuals,measurement_type):
    import numpy as np
    n = np.size(residuals)
    mean = (1/n)*(np.sum(residuals))
    variance = (1/n)*(np.sum(np.square(residuals)))
    rms = np.sqrt(variance)
    rms_about_zero = np.sqrt((n/(n-1))*variance)
    
                
#     print('mean            ',measurement_type ,':',mean)
#     print('rms             ',measurement_type ,':',rms)
#     print('rms about zero  ',measurement_type ,':',rms_about_zero)
#     print()
    
    return(mean,rms,rms_about_zero)






In [ ]:
%load_ext autoreload
%autoreload 2


fig = make_subplots(
    rows=1, cols=1,
    subplot_titles=(['Sampled Orbit Densities']),
    vertical_spacing = 0.15)


fig = plot_density_orbit_avg(fig, Obj_Geodyn['dtm2020_p1']   ,0)
fig = plot_density_orbit_avg(fig, Obj_Geodyn['dtm2020_p2']   ,0)
fig = plot_density_orbit_avg(fig, Obj_Geodyn['jb2008_p1']   ,1)
fig = plot_density_orbit_avg(fig, Obj_Geodyn['jb2008_p2']   ,1)
fig = plot_density_orbit_avg(fig, Obj_Geodyn['msis2_p1']   ,2)
fig = plot_density_orbit_avg(fig, Obj_Geodyn['msis2_p2']   ,2)


font_dict=dict(family='sans-serif',size=16,color='black')
fig.update_xaxes(showline=True,
                 showticklabels=True,
                 linecolor='black',
                 linewidth=1,
                 ticks='inside',
                 tickfont=font_dict,
                 mirror='allticks',
                 tickwidth=1.4,
                 tickcolor='black',
                 gridcolor='lightgray')


# fig.update_yaxes(range=[1.15, 1.35], row=1,col=1)
# fig.update_yaxes(range=[1.15, 1.35], row=1,col=2)
fig.update_yaxes(showline=True,      # add line at x=0
                 linecolor='black',  # line color
                 linewidth=1,      # line size
                 ticks='inside',    # ticks outside axis
                 tickfont=font_dict, # tick label font
                 mirror='allticks',  # add ticks to top/right axes
                 tickwidth=1.4,      # tick width
                 tickcolor='black',  # tick color
                  gridcolor='lightgray')

fig.update_annotations(font_size=18)  # Increase size of subplot title
fig.update_layout(
#                   title = '',
                  autosize=True,#    width=1000,    height=450,
                  legend= {'itemsizing': 'constant'},
                  font=font_dict,
                  plot_bgcolor='white', 
                 )
fig.show(config=config)



### Total RMS

In [ ]:


def plot_TotalRMS_NTW(fig, obj_m1, plotname, plot_num):


    ####  Get plot Parameters for this model
    model_m1 = obj_m1.__dict__['global_params']['den_model']
    col,x_annot,y_annot1,y_annot2,m_size = get_plot_params(plot_num, model_m1)
    
    doy_dates = []
    rms_totals = []
    for i in obj_m1.__dict__['global_params']['arc_input']:
        doy_dates.append(pd.to_datetime(datetime.datetime(int(i.split('.')[0]), 1, 1) + datetime.timedelta(int(i.split('.')[1]) - 1)))
        rms_totals.append(obj_m1.__dict__['Statistics'][i]['RMS_total_NTW'].values[0])


    fig.add_trace(go.Scattergl(
                         x=doy_dates,
                         y=rms_totals,
                     name='NTW',
                     mode='markers+lines',
                     marker=dict(color=col,size=10),
                     showlegend=False)),

    fig = legend_as_annotation(fig, plotname, col, x_annot, y_annot2)
    fig.update_yaxes( title="Total RMS [meters]" ,type="linear" , exponentformat= 'power')
    fig.update_xaxes( title="Date"    )


    return(fig)
    


In [ ]:
%load_ext autoreload
%autoreload 2

fig = make_subplots(
    rows=1, cols=1,
    subplot_titles=(['Total RMS in the NTW Coordinate System']),
    vertical_spacing = 0.1)


fig = plot_TotalRMS_NTW(fig, Obj_Geodyn['dtm2020_p1'] , 'dtm2020' , 0)
fig = plot_TotalRMS_NTW(fig, Obj_Geodyn['dtm2020_p2'] , 'dtm2020' , 0)
fig = plot_TotalRMS_NTW(fig, Obj_Geodyn['jb2008_p1']  , 'jb2008'  , 1)
fig = plot_TotalRMS_NTW(fig, Obj_Geodyn['jb2008_p2']  , 'jb2008'  , 1)
fig = plot_TotalRMS_NTW(fig, Obj_Geodyn['msis2_p1']   , 'msis2'   , 2)
fig = plot_TotalRMS_NTW(fig, Obj_Geodyn['msis2_p2']   , 'msis2'   , 2)


font_dict=dict(family='sans-serif',size=16,color='black')
fig.update_xaxes(showline=True,
                 showticklabels=True,
                 linecolor='black',
                 linewidth=1,
                 ticks='inside',
                 tickfont=font_dict,
                 mirror='allticks',
                 tickwidth=1.4,
                 tickcolor='black',
                 gridcolor='lightgray')


# fig.update_yaxes(range=[1.15, 1.35], row=1,col=1)
# fig.update_yaxes(range=[1.15, 1.35], row=1,col=2)
fig.update_yaxes(showline=True,      # add line at x=0
                 linecolor='black',  # line color
                 linewidth=1,      # line size
                 ticks='inside',    # ticks outside axis
                 tickfont=font_dict, # tick label font
                 mirror='allticks',  # add ticks to top/right axes
                 tickwidth=1.4,      # tick width
                 tickcolor='black',  # tick color
                  gridcolor='lightgray')

fig.update_annotations(font_size=18)  # Increase size of subplot title
fig.update_layout(
#                   title = '',
                  autosize=True,#    width=1000,    height=450,
                  legend= {'itemsizing': 'constant'},
                  font=font_dict,
                  plot_bgcolor='white', 
                 )
fig.show(config=config)

### In track residuals

In [ ]:
def PLOT__intrack_residuals_w_rms(fig, obj_m1, plot_num):
    
    
    ####  Get plot Parameters for this model
    model_m1 = obj_m1.__dict__['global_params']['den_model']
    col,x_annot,y_annot1,y_annot2,m_size = get_plot_params(plot_num, model_m1)
    

    mean_tot = []     
    model_m1 = obj_m1.__dict__['global_params']['den_model']
    for ii,arc in enumerate(obj_m1.__dict__['global_params']['arc_input']):
        
        data_resids = obj_m1.__dict__['OrbitResids'][arc]['resids']
#         low_idx, high_idx = hl_envelopes_idx(data_resids['T'])

        fig.add_trace(go.Scattergl(x=data_resids['Date'][::10],
                                   y=data_resids['T'][::10],
                                     name= model_m1+arc,
                                     mode='markers',
                                     opacity=0.8,
                                     marker=dict(
                                         color=col,
                                         size=m_size,),
                                     showlegend=False,
                                     ),
                                     secondary_y=False,
                                     row=1, col=1,
                                 )


        (mean,rms,rms_about_zero) = STATS_residuals(data_resids['T'], 'in-track')
        mean_tot.append(mean)
                                                                                                        

    for ii,arc in enumerate(obj_m1.__dict__['global_params']['arc_input']):
        fig = add_stats_annotation(fig, model_m1+'<br>Mean='+ str(np.round(np.mean(mean_tot),4))+'<br>RMS='+ str(np.round(obj_m1.__dict__['Statistics'][arc]['T_RMS'].mean(),4)), col , x_annot, y_annot1)

    fig.update_yaxes( title="Residuals (m)",exponentformat= 'power',row=1, col=1)
    fig.update_xaxes( title="Date", row=1, col=1)


    return(fig)


In [ ]:
%load_ext autoreload
%autoreload 2


fig = make_subplots(
    rows=1, cols=1,
    subplot_titles=(['In-Track Residuals']),
    vertical_spacing = 0.1)

fig = PLOT__intrack_residuals_w_rms(fig, Obj_Geodyn['msis2_p1']     , 4)
fig = PLOT__intrack_residuals_w_rms(fig, Obj_Geodyn['msis2_p2']     , 4)

fig = PLOT__intrack_residuals_w_rms(fig, Obj_Geodyn['dtm2020_p1']    , 0)
fig = PLOT__intrack_residuals_w_rms(fig, Obj_Geodyn['dtm2020_p2']    , 0)
fig = PLOT__intrack_residuals_w_rms(fig, Obj_Geodyn['jb2008_p1']    , 2)
fig = PLOT__intrack_residuals_w_rms(fig, Obj_Geodyn['jb2008_p2']    , 2)


font_dict=dict(family='sans-serif',size=16,color='black')
fig.update_xaxes(showline=True,
                 showticklabels=True,
                 linecolor='black',
                 linewidth=1,
                 ticks='inside',
                 tickfont=font_dict,
                 mirror='allticks',
                 tickwidth=1.4,
                 tickcolor='black',
                 gridcolor='lightgray')


# fig.update_yaxes(range=[1.15, 1.35], row=1,col=1)
# fig.update_yaxes(range=[1.15, 1.35], row=1,col=2)
fig.update_yaxes(showline=True,      # add line at x=0
                 linecolor='black',  # line color
                 linewidth=1,      # line size
                 ticks='inside',    # ticks outside axis
                 tickfont=font_dict, # tick label font
                 mirror='allticks',  # add ticks to top/right axes
                 tickwidth=1.4,      # tick width
                 tickcolor='black',  # tick color
                  gridcolor='lightgray')

fig.update_annotations(font_size=18)  # Increase size of subplot title
fig.update_layout(
#                   title = '',
                  autosize=True,#    width=1000,    height=450,
                  legend= {'itemsizing': 'constant'},
                  font=font_dict,
                  plot_bgcolor='white', 
                 )
fig.show(config=config)

## additional plot variations

### Density and F10.7

In [ ]:
from netCDF4 import Dataset
def read_nc_file( filename, variables):
    ''' This function reads the TIEGCM .nc files and saves the given input variables to a dictionary.
        The breakloop feature is here so that if the file doesn't exist the code can still continue.  '''
    status = os.path.exists(filename)
    
    if status == True:
        data = {}
        for i, var_names in enumerate(variables):
            ncid =  Dataset(filename,"r+", format="NETCDF4")# filename must be a string
            varData = ncid.variables
            data[var_names] = np.array(varData[var_names])  
    elif status == False:
        print('No File Found', filename )
        breakloop = True
        data = 0
        return( data , breakloop)
    breakloop = False
    return(data,breakloop )





arc_list = [        2018287,
                    #
                    2018292,
                    2018293,
                    2018294,
                    2018295,
                    2018296,
                    2018297,
                    2018298,
                    2018299,                   
                    #
                    2018304,
                    2018305,
                    2018306,
                    2018307,
                    2018308,
                    #
                    2018313,
                    2018314,
                    2018315,
                    2018316,
                    2018317,
                    2018318,
                    2018319,
                    2018320,
                    2018321,
                    2018322,
                    2018323,
                    2018324,
                    2018325,
                    2018326,
                    2018327,
                    #
                    2018335,
                    2018336,
                    2018337,
                    #
                    2018349,
                    2018350,
                    2018351,
                    2018352,
                    #
                    2018356,
                    2018357,
                    2018358,
                    #
                    2018365,
                    2019001,
                    2019002,
                    2019003,
                    2019004,
                    2019005,
                    2019006,
                    2019007,
                    2019008,
                    ]

path_to_f107 = '/data/data_geodyn/gpi_1960001-2021243_f107aDaily.nc'
variables = ['year_day', 'f107d', 'f107a', 'kp']
f107_data = read_nc_file(path_to_f107, variables)

date = []
kp_list = []
f107d_list = []
f107a_list  = []
date_3hr = []


for i,val in enumerate(arc_list):
    
    index = f107_data[0]['year_day']==val
    kp_list.append(f107_data[0]['kp'][index][0])
    f107d_list.append(f107_data[0]['f107d'][index][0])
    f107a_list.append(f107_data[0]['f107a'][index][0])
    
    date.append(pd.to_datetime( str(val), format='%Y%j'))

    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=0))
    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=3))
    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=6))
    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=9))
    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=12))
    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=15))
    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=18))
    date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=21))
#     date_3hr.append(pd.to_datetime( str(val), format='%Y%j') +pd.Timedelta(hours=24))
    
kp_expand = []
for i in kp_list:
    for ii in i:
        kp_expand.append(ii)
        
        
        
solar_fluxes = {}

solar_fluxes['f107d_list'] = f107d_list
solar_fluxes['f107a_list'] = f107a_list
solar_fluxes['date']       = date
solar_fluxes['date_3hr']   = date_3hr
solar_fluxes['kp_expand']  = kp_expand
        
        
        
def plot_density_orbit_avg_solarflux(fig, obj_m1, plot_num, solar_fluxes ):
    

    ####  Get plot Parameters for this model
    model_m1 = obj_m1.__dict__['global_params']['den_model']
    col,x_annot,y_annot1,y_annot2,m_size = get_plot_params(plot_num, model_m1)
    
    for ii,arc in enumerate(obj_m1.__dict__['global_params']['arc_input']):
        
        vals  = np.arange(obj_m1.__dict__['Density'][arc].index[0],obj_m1.__dict__['Density'][arc].index[-1]+1)
        df = obj_m1.__dict__['Density'][arc].set_index('Date',drop=False ) 
        df['i_vals'] = vals
        index_date = df.loc[df.index.max()]['i_vals'].min()

        
        time_avg,d_avg, d_avg_rolling = orb_avg(obj_m1.Density, arc)
        
        
        fig.add_trace(go.Scattergl(x=time_avg,
                                 y=d_avg_rolling,
#                                  y=d_avg,
                                name= model_m1 ,
                                mode='markers',
                                marker=dict(
                                color=col,
                                size=7,),
                                showlegend=False,
                                   ),
                                   row=1, col=1,
                                   )
        

        fig.update_yaxes(type="log", exponentformat= 'power',row=1, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_yaxes(title_text="kg/m^3", row=1, col=1)
#     fig.update_yaxes(title_text="nT", row=2, col=1)
#     fig.update_yaxes(title_text="sfu", row=3, col=1)
    fig.update_layout(legend= {'itemsizing': 'constant'})
#     fig.update_layout(
#         font=dict(          size=18, ),
#         autosize=False,
#         width=900,
#         height=1000,)
    
    return(fig)


In [ ]:
np.shape(date_3hr)

In [ ]:
np.shape(kp_expand)